In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install dwave-system
!pip install pandas-datareader --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 16.6 MB/s 
     |████████████████████████████████| 2.3 MB 74.7 MB/s 
     |████████████████████████████████| 111 kB 70.8 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 5.6 MB 99.5 MB/s 
     |████████████████████████████████| 15.9 MB 323 kB/s 
     |████████████████████████████████| 44 kB 3.4 MB/s 
     |████████████████████████████████| 6.1 MB 669 kB/s 
     |████████████████████████████████| 282 kB 84.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 13.8 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [3]:
from ast import Return
from itertools import product
import json
import random
import numpy as np
import pandas as pd
import os
import sys
import time
import yaml
from pandas_datareader.data import DataReader
import datetime as dt
from dimod import Integer, Binary
from dimod import quicksum
from dimod import ConstrainedQuadraticModel, DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler, LeapHybridCQMSampler
from dimod import ExactSolver, ExactCQMSolver
import gc
import matplotlib
import seaborn as sns
from itertools import takewhile, count
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_model import ARMAResults 
from IPython.display import clear_output
from typing import NamedTuple

try:
    import matplotlib.pyplot as plt
except ImportError:
    matplotlib.use("agg")
    import matplotlib.pyplot as plt

pd.set_option('max_row', None)
pd.set_option('max_column', None)


class SinglePeriod:
    def __init__(
        self, 
        data,  
        preds, 
        budget, 
        init_budget,
        sampler_args, 
        dates,
        model_type,
        alpha,
        verbose,
        t_cost,
        label,
        init_holdings):
    
        self.df = data
        self.preds = preds
        self.budget = budget
        self.init_budget = init_budget
        self.dates = dates
        self.model_type = model_type
        self.verbose = verbose
        self.t_cost = t_cost
        self.label = label
        self.init_budget = budget
        self.stocks = self.df.columns.tolist()
        
        if init_holdings is not None:
            self.init_holdings = init_holdings
        else:
            self.init_holdings = {s:0 for s in self.stocks}
  
        if isinstance(alpha, (list, tuple)):
            self.alpha = alpha[0]
            self.alpha_list = list(alpha)
        elif isinstance (alpha, (int, float)):
            self.alpha = alpha
        else:
            raise TypeError("Use integer or float for alpha, or a list or tuple of either type.")

        self.model = {'CQM': None, 'DQM': None}

        self.sample_set = {}
        if sampler_args:
            self.sampler_args = json.loads(str(sampler_args))
        else:
            self.sampler_args = {}

        self.sampler = {'CQM': LeapHybridCQMSampler(**self.sampler_args), 'Exact': ExactCQMSolver()}
        self.solution = {}
        self.precision = 2

        self.list_dates = self.df.index.values

    def build_cqm(self, idx, max_risk=None, min_return=None):
        # Instantiating the CQM object
        cqm = ConstrainedQuadraticModel()
        self.max_num_shares = (self.budget/self.df.loc[idx,:]).astype(int)

        # Defining and adding variables to the CQM model
        x = {s: Integer("%s" %s, lower_bound=0, upper_bound=self.max_num_shares[s]) for s in self.stocks}

        print(f'idx: {idx}')
        print(f'self.df')
        display(self.df.head())
        print(f'self.df.index.values: {self.df.index.values}')
        print(f'self.list_dates: {self.list_dates}')
        self.position = np.where(self.list_dates == idx)[0][0]
        print(f'self.position: {self.position}')
        #self.position2 = np.where(self.list_dates == idx)[0]
        #print(f'self.position 2: {self.position2}')
        self.initial_position = self.list_dates[self.position-10]
        print(f'self.initial_position: {self.initial_position}')
        #print('self.df')
        #display(self.df)
        self.price = self.df.loc[idx, :]
        print(f'sum price: {self.price.sum()}')
        #print('price')
        #print(self.price)
        self.pct_returns = self.df.pct_change()
        self.daily_returns = self.pct_returns.loc[self.initial_position:idx, :]
        print(f'sum daily returns: {self.daily_returns.stack().sum()}')
        self.avg_daily_returns = self.daily_returns.rolling(window=1).mean().loc[idx,:]
        #self.avg_daily_returns = self.preds
        print(f'avg daily returns: {self.avg_daily_returns.sum()}')
        # print('avg daily returns 2')
        # display(self.avg_daily_returns2.head())
        self.covariance_matrix = self.daily_returns.cov()
        self.correlation_matrix = self.daily_returns.corr()
        print('cov matrix')
        display(self.covariance_matrix.head())
        print(f'covariance_matrix sum: {self.covariance_matrix.stack().sum()}')
        print('corr matrix')
        display(self.correlation_matrix.head())
        print(f'correlation_matrix sum: {self.correlation_matrix.stack().sum()}')

        
        # Defining risk expression
        risk = 0
        for s1, s2 in product(self.stocks, self.stocks):
            coeff = (self.covariance_matrix[s1][s2] * self.price[s1] * self.price[s2])
            risk = risk + coeff*x[s1]*x[s2]       
        print(f'risk: {risk}')

        returns = 0
        for s in self.stocks:
            returns = returns + self.price[s] * self.avg_daily_returns[s] * x[s]
        print(f'returns: {returns}')

        if not self.t_cost:
            print('t_cost False')
            cqm.add_constraint(quicksum([x[s]*self.price[s] for s in self.stocks]) <= self.budget, label='upper_budget')
            cqm.add_constraint(quicksum([x[s]*self.price[s] for s in self.stocks]) >= 0.997*self.budget, label='lower_budget')
        else:
            print('t_cost True')
            # Modeling transaction cost
            x0 = self.init_holdings
            # y indicates stock allocations that changed
            y = {s: Binary("Y[%s]" %s) for s in self.stocks}

            lhs = 0

            for s in self.stocks:
                lhs = lhs + 2*self.t_cost*self.price[s]*x[s]*y[s] + self.price[s]*(1 - self.t_cost)*x[s] \
                      - 2*self.t_cost*self.price[s]*x0[s]*y[s] - self.price[s]*(1 - self.t_cost)*x0[s]

            print(f't_cost: {self.t_cost}')
            print(f'budget: {self.budget}')
            print(f'init_budget: {self.init_budget}')

            cqm.add_constraint(lhs <= self.budget, label='upper_budget')
            cqm.add_constraint(lhs >= self.budget - 0.003*self.init_budget, label='lower_budget')

            # indicator constraints
            for s in self.stocks:
                cqm.add_constraint(x[s] - x0[s]*y[s] >= 0, label=f'indicator_constraint_gte_{s}')
                cqm.add_constraint(x[s] - x[s]*y[s] <= x0[s], label=f'indicator_constraint_lte_{s}')

           
        if max_risk:
            # Adding maximum risk constraint
            cqm.add_constraint(risk <= max_risk, label='max_risk')

            # Objective: maximize return
            cqm.set_objective(-1*returns)
        elif min_return:
            # Adding minimum returns constraint
            cqm.add_constraint(returns >= min_return, label='min_return')

            # Objective: minimize risk
            cqm.set_objective(risk)
        else:
            # Objective: minimize mean-variance expression
            print('cqm debugging')
            print(f'risk: {type(risk)}')
            print(f'returns: {type(returns)}')
            print(returns)
            
            cqm.set_objective(self.alpha*risk - returns)

        cqm.substitute_self_loops()

        self.model['CQM'] = cqm

    def solve_cqm(self, idx, max_risk=None, min_return=None):
        self.build_cqm(idx, max_risk, min_return)

        if self.model_type == 'CQM':
            self.sample_set['CQM'] = self.sampler['CQM'].sample_cqm(self.model['CQM'], label=self.label)
        else:        
            gc.collect()
            self.sample_set['CQM'] = self.sampler['Exact'].sample_cqm(self.model['CQM'])

        n_samples = len(self.sample_set['CQM'].record)
        print(f'n_samples: {n_samples}')
        feasible_samples = self.sample_set['CQM'].filter(lambda d: d.is_feasible)

        if not feasible_samples:
            raise Exception("No feasible solution could be found for this problem instance.")
        else:
            best_feasible = feasible_samples.first

            solution = {}
            solution['stocks'] = {k:int(best_feasible.sample[k]) for k in self.stocks}
            print(f'solution_stocks: {solution["stocks"]}')
            solution['return'], solution['risk'] = self.compute_risk_and_returns(solution['stocks'])
            print(f'solution_return: {solution["return"]}')
            print(f'solution_risk: {solution["risk"]}')
            spending = sum([self.price[s]*max(0, solution['stocks'][s] - self.init_holdings[s]) for s in self.stocks])
            print(f'spending: {spending}')
            sales = sum([self.price[s]*max(0, self.init_holdings[s] - solution['stocks'][s]) for s in self.stocks])
            print(f'sales: {sales}')

            transaction = self.t_cost*(spending + sales)
            print(f'transaction: {transaction}')

            if self.verbose:
                print(f'Number of feasible solutions: {len(feasible_samples)} out of {n_samples} sampled.')
                print(f'\nBest energy: {self.sample_set["CQM"].first.energy: .2f}')
                print(f'Best energy (feasible): {best_feasible.energy: .2f}')

            print(f"\nEstimated Returns: {solution['return']}")
            print(f"Sales Revenue: {sales:.2f}")
            print(f"Purchase Cost: {spending:.2f}")
            print(f"Transaction Cost: {transaction:.2f}")
            print(f"Variance: {solution['risk']}\n")

            print(f'solution: {solution}')
            return solution


    def compute_risk_and_returns(self, solution):
        """Compute the risk and return values of solution.
        """
        for s1 in solution:
            print(f's1: {s1}, price: {self.price[s1]}')
        print(f'covariance matrix: {self.covariance_matrix.stack().mean()}')
        
        variance = 0.0
        for s1, s2 in product(solution, solution):
            variance += (solution[s1] * self.price[s1] * solution[s2] * self.price[s2] * self.covariance_matrix[s1][s2])

        est_return = 0
        for stock in solution:
            est_return += solution[stock]*self.price[stock]*self.avg_daily_returns[stock]

        return round(est_return, 2), round(variance, 2)

    def run(self, min_return=0, max_risk=0):

        return self.solve_cqm(min_return=min_return, max_risk=max_risk)

In [4]:
class MultiPeriod():
    """Solve the multi-period (dynamic) portfolio optimization problem.
    """
    def __init__(self, 
                data, 
                baseline,
                budget,
                sampler_args,
                dates,
                file_path,
                model_type,
                alpha,
                verbose,
                t_cost,
                label,
                init_holdings,
                save_path):
    
        self.save_path = save_path
        self.df = data
        self.stocks = self.df.columns.values.tolist()
        
        self.sampler_args = sampler_args
        self.budget = budget
        self.sampler_args = sampler_args
        self.verbose = verbose
        self.label = label
        self.t_cost = t_cost
        self.baseline = baseline
        self.dates = dates
        self.model_type = model_type

        initial_gspc = self.baseline.query('Date==@dates[0]').values.tolist()[0][0]
        #print('initial baseline')
        #display(self.baseline)
        #print(f'initial_gspc: {initial_gspc}')
        self.baseline.iloc[:,0] = (self.baseline.iloc[:,0] / initial_gspc - 1)
        self.init_budget = budget
        #print('final baseline')
        #display(self.baseline)
        #print('initial data')
        #display(self.df.head())

        if init_holdings is not None:
            self.init_holdings = init_holdings
        else:
            self.init_holdings = {s:0 for s in self.stocks}

        print('iniital_holdings')
        print(self.init_holdings)
  
        if isinstance(alpha, (list, tuple)):
            self.alpha = alpha[0]
            self.alpha_list = list(alpha)
        elif isinstance (alpha, (int, float)):
            self.alpha = alpha
        else:
            raise TypeError("Use integer or float for alpha, or a list or tuple of either type.")

        print('init holdings keys')
        print(set(self.init_holdings.keys()))
        self.stocks = list(set(self.df.columns.values.tolist()).intersection(set(self.init_holdings.keys())))
        self.df = self.df.loc[:, self.stocks].copy()
        self.active_data = self.df.loc[dates[0]:,:]
        self.result = pd.DataFrame(columns=self.stocks)
        self.sample_set = {}
        self.sampler_args = sampler_args
        self.solution = {}
        
    def run(self, max_risk=0, min_return=0, num=0):
        """Solve the rebalancing portfolio optimization problem.
        Args:
            max_risk (int): Maximum risk for the CQM risk bounding formulation.
            min_return (int): Minimum return for the CQM return bounding formulation.
        """
        
        num_days = self.active_data.shape[0]
        print(f'num_days: {num_days}')
        first_purchase = True
        #result = {}
        #baseline_result = {}
        #days = []
        
        # dataframe to save output data
        headers = self.stocks
        self.output = pd.DataFrame(columns=headers)
        self.agg_results = pd.DataFrame(columns=['idx', 'variance', 'returns', 'value'])
        #row = []
        

        for idx, row in self.active_data.iterrows():
            print(f'idx: {idx}')
            df_train = self.df.loc[:idx, :]
            #print('df train')
            #display(df_train.head())
            self.preds = predict_returns(df_train)
            self.price = self.df.loc[idx, :]
            baseline_current = self.baseline.loc[idx, :]
            single_period = SinglePeriod(data=self.df, 
                                         preds=self.preds.pct_change().iloc[-1,:], 
                                         budget=self.budget, 
                                         init_budget = self.init_budget,
                                         sampler_args=self.sampler_args, 
                                         dates=self.dates,
                                         model_type=self.model_type,
                                         alpha=self.alpha,
                                         verbose=self.verbose,
                                         t_cost=self.t_cost,
                                         label=self.label,
                                         init_holdings=self.init_holdings)

            if first_purchase:
                #initial_budget = self.budget
                print(f'first purchase budget: {self.budget}, initial_budget: {self.init_budget}')
                baseline_shares = (self.budget / baseline_current)
                display(self.baseline)
                self.solution = single_period.solve_cqm(idx, max_risk=max_risk, min_return=min_return)
                self.init_holdings = self.solution['stocks']
            else:
                #self.budget = sum([self.df.iloc[-1][s]*self.solution['stocks'][s] for s in self.stocks])
                self.budget = sum([self.price[s]*self.solution['stocks'][s] for s in self.stocks])
                print(f'new purchase budget: {self.budget}, initial_budget: {self.init_budget}')
                # if self.t_cost and not first_purchase:
                #     self.budget = 0
                self.solution = single_period.solve_cqm(idx, max_risk=max_risk, min_return=min_return)
                self.init_holdings = self.solution['stocks']
                print(f'new purchase final budget: {self.budget}, initial_budget: {self.init_budget}')

            value = sum([self.price[s]*self.solution['stocks'][s] for s in self.stocks])
            print(f'value: {value}')
            print(f'sum shares: {sum([self.solution["stocks"][s] for s in self.stocks])}')
            returns = self.solution['return']
            variance = self.solution['risk']

            self.output = pd.concat([self.output, pd.DataFrame([{s: self.solution['stocks'][s] for s in self.stocks}])])
            self.agg_results = pd.concat([self.agg_results, pd.DataFrame([{'idx': idx, 'variance': variance, 'returns': returns, 'value': value}])])
            
            first_purchase = False

            #clear_output(wait=True)
            initial_budget = self.agg_results['value'].values.tolist()[0]
            self.agg_results['value_norm'] = self.agg_results['value'] / initial_budget - 1
            display(self.agg_results)
            
            fig = plt.figure(figsize=(10.708,8.2))
            sns.scatterplot(x='Date', y='^GSPC', data=self.baseline, label='Fund Portfolio')
            sns.scatterplot(x='idx', y='value_norm', data=self.agg_results, label='Quantum')
            plt.show()
            print(f'\nRun completed.\n')

            self.output.to_pickle(self.save_path+'output.pkl')
            self.agg_results.to_pickle(self.save_path+'agg_results.pkl')
            self.baseline.to_pickle(self.save_path+'baseline.pkl')

        return None

In [5]:
def load_data(initial_date, dates, stocks, baseline):
    print(f"\nLoading live data from the web from Yahoo! finance", f"from {initial_date} to {dates[1]}...")

    # Read in daily data; resample to monthly
    print(f'reader initial date: {initial_date}')
    print(f'reader final_date: {dates[1]}')
    panel_data = DataReader(stocks, 'yahoo', initial_date, dates[1])
    df = pd.DataFrame(index=panel_data.index, columns=stocks)
    
    for i in stocks:
        df[i] = panel_data[[('Adj Close',  i)]]
    df.dropna(axis=1, inplace=True)

    # Read in baseline data; resample to monthly
    index_df = DataReader(baseline, 'yahoo', dates[0], dates[1])
    df_baseline = pd.DataFrame(index=index_df.index, columns=baseline)
    for i in baseline:
        df_baseline[i] = index_df[[('Adj Close',  i)]]

    return df, df_baseline

In [6]:
def predict_returns(df):
    result = {}
    df = df.copy()

    for i, column in enumerate(df.columns.values):
        train = df.loc[:, column]
        history = [x for x in train.values]
        pred = list()
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit()
        output = model_fit.forecast()
        result[column] = output[0]

    pred_df = pd.DataFrame([result])
    return pd.concat([df, pred_df])

In [7]:
# GA block
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if np.random.rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = np.random.randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            inverse_s = ''

            for b in bitstring[i]:  
                if b == '0':
                    inverse_s += '1'       
                else:
                    inverse_s += '0'
            bitstring[i] = inverse_s
    return bitstring

def selection(pop, scores, k=3):
	# first random selection
	selection_ix = np.random.randint(len(pop))
	#print(selection_ix)
	for ix in np.random.randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	#print(f'selection ix: {selection_ix}')
	return pop[selection_ix]

def objective(x, daily_return):
    offset = 0
    #offset = 1000 if any([y<0 for y in x]) else 0
    return -1 * sum([d1 * d2 for d1,d2 in zip(x, list(daily_return))]) + offset

def sample_initial_pop(prices, n_pop, budget):
    pop = []
    total = 0
    #shares = np.random.randint(100, size=len(prices))
    score_report = []
    for i in range(n_pop):
        while True:
            factor = np.random.randint(10, size=1)
            shares = [int(factor[0] * i) for i in np.random.chisquare(1,len(prices))]
            #print(f'len shares: {len(shares)}')
            #print(f'len prices: {len(prices)}')
            #print(f'shares: {shares}')
            #print(f'prices: {prices}')
            #print('list zip')
            #print([d1 * int(d2) for d1, d2 in zip(prices, shares)])
            total = sum([d1 * int(d2) for d1, d2 in zip(prices, shares)])
            if total <= budget and total > 0.9 * budget: break
        score_report.append(total)
        pop.append(shares)
    pop = np.array(pop)
    fig, ax = plt.subplots(figsize=(12, 8)) 
    ax.hist(score_report) 
    ax.set_title('scores')
    plt.show()
    return pop


def generate_initial_state(df, n_iter, r_cross, r_mut, n_pop, budget):
    prices = df.iloc[0, :].values.tolist()
    #print(f'prices: {prices}')
    pop = sample_initial_pop(prices, n_pop, budget)
    daily_return = df.pct_change().iloc[-1, :].values
    # keep track of best solution
    best, best_eval, best_total = 0, objective(pop[0], daily_return), 0   
    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(c, daily_return) for c in pop]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best_total = sum([d1 * d2 for d1, d2 in zip(prices, pop[i])])
                best, best_eval  = pop[i], scores[i]
    
        selected = [selection(pop, scores).tolist() for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
        # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
        # crossover and mutation
            for c in crossover(p1, p2, r_cross): #.block_until_ready():
                # mutation
                m = mutation(["{0:b}".format(i) for i in c], r_mut)
                cm = [int(i, 2) for i in m]
                children.append(cm)

        # replace population
        pop = np.array(children)
        total = sum([d1 * d2 for d1, d2 in zip(prices, scores)])
        if gen % 10 == 0:
            total = np.dot(np.array(prices), np.array(pop).T)
            clear_output(wait=True)
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 8)) 
            ax[0].hist(scores) 
            ax[0].set_title('scores')
            ax[1].hist(total)
            ax[1].set_title('budget')
            plt.show()
            print(f'{gen} best eval:{best_eval}')
            print(f'mean score: {np.mean(scores)}')
            print(f'best total: {best_total}')

    print(f'best: {best}')
    print(f'best_eval: {best_eval}')
    print(f'best_total: {best_total}')
    names = df.columns.tolist()
    df = pd.DataFrame([[i,j] for i,j in zip(names, best)], columns=['a', 'b'])
    return df, best_total

In [8]:
load_path = '/content/drive/MyDrive/dwave/conf/'
pickle_file = '/content/drive/MyDrive/ga/df.pkl'
with open(load_path+'config.txt', 'r') as conf:
    cred = conf.readline().strip()

cfg = {'stocks': None,
            'dates': ["2021-06-01", "2021-07-01"],
            'initial_date': "2021-01-01",
            'budget': 1000000,
            'bin_size': '',
            'gamma': '',
            'alpha': 0.005,
            'file_path': None,
            'baseline': ['^GSPC'],
            'max_risk': 0.0,
            'min_return': 0.0,
            'model_type': 'CQM',
            'rebalance': True,
            'sampler_args': cred,
            'verbose': False,
            'num': 0,
            't_cost': 0.01,
            'n_iter': 10000,
            'r_cross': 0.8,
            'r_mut': 0.01,
            'n_pop': 1000,
            'label': 'Run97',
            'init_holdings_path': '/content/drive/MyDrive/dwave/init_holdings.pkl',
            'calculation_type': 'daily',
            'save_path': '/content/drive/MyDrive/dwave/'
}
locals().update(cfg)

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
stocks = first_table['Symbol'].values.tolist()

df, df_baseline = load_data(initial_date, dates, stocks, baseline)
#print('initial df')
#display(df.head())

# if not init_holdings_path: 
#     init_holdings_df, budget = generate_initial_state(df, n_iter, r_cross, r_mut, n_pop, budget)
#     init_holdings_df.to_pickle(save_path+'init_holdings.pkl')
# else:
#     init_holdings_df = pd.read_pickle(init_holdings_path)
#     prices = df.loc[dates[0], :].values.tolist()
#     print('init holdings df')
#     display(init_holdings_df.head())
#     print(f'prices: {prices}')
#     pop = init_holdings_df['b'].values.tolist()
#     print(f'pop: {pop}')
#     budget = sum([d1 * d2 for d1, d2 in zip(prices, pop)])
#     print(f'budget: {budget}')

# init_holdings = {}
# for idx, row in init_holdings_df.iterrows():
#     init_holdings.update({row['a']: row['b']})
init_holdings = {s:0 for s in stocks}

#print(f'init_holdings: {init_holdings}')
#print(f'new budget: {budget}')
  
# checking if the directory demo_folder 
# exist or not.
if not os.path.exists(save_path+label):
    os.makedirs(save_path+label)

my_portfolio = MultiPeriod(data=df,
                        baseline=df_baseline,
                        budget=budget,
                        sampler_args=sampler_args,
                        dates=dates,
                        file_path=file_path,
                        model_type=model_type,
                        alpha=alpha,
                        verbose=verbose,
                        t_cost=t_cost,
                        label=label,
                        init_holdings=init_holdings,
                        save_path=save_path+label+'/'
                        )

my_portfolio.run(min_return=min_return, max_risk=max_risk, num=num)


Loading live data from the web from Yahoo! finance from 2021-01-01 to 2021-07-01...
reader initial date: 2021-01-01
reader final_date: 2021-07-01


/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'BRK.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'BF.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'CEG', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


iniital_holdings
{'MMM': 0, 'AOS': 0, 'ABT': 0, 'ABBV': 0, 'ABMD': 0, 'ACN': 0, 'ATVI': 0, 'ADM': 0, 'ADBE': 0, 'ADP': 0, 'AAP': 0, 'AES': 0, 'AFL': 0, 'A': 0, 'APD': 0, 'AKAM': 0, 'ALK': 0, 'ALB': 0, 'ARE': 0, 'ALGN': 0, 'ALLE': 0, 'LNT': 0, 'ALL': 0, 'GOOGL': 0, 'GOOG': 0, 'MO': 0, 'AMZN': 0, 'AMCR': 0, 'AMD': 0, 'AEE': 0, 'AAL': 0, 'AEP': 0, 'AXP': 0, 'AIG': 0, 'AMT': 0, 'AWK': 0, 'AMP': 0, 'ABC': 0, 'AME': 0, 'AMGN': 0, 'APH': 0, 'ADI': 0, 'ANSS': 0, 'AON': 0, 'APA': 0, 'AAPL': 0, 'AMAT': 0, 'APTV': 0, 'ACGL': 0, 'ANET': 0, 'AJG': 0, 'AIZ': 0, 'T': 0, 'ATO': 0, 'ADSK': 0, 'AZO': 0, 'AVB': 0, 'AVY': 0, 'BKR': 0, 'BALL': 0, 'BAC': 0, 'BBWI': 0, 'BAX': 0, 'BDX': 0, 'WRB': 0, 'BRK.B': 0, 'BBY': 0, 'BIO': 0, 'TECH': 0, 'BIIB': 0, 'BLK': 0, 'BK': 0, 'BA': 0, 'BKNG': 0, 'BWA': 0, 'BXP': 0, 'BSX': 0, 'BMY': 0, 'AVGO': 0, 'BR': 0, 'BRO': 0, 'BF.B': 0, 'CHRW': 0, 'CDNS': 0, 'CZR': 0, 'CPT': 0, 'CPB': 0, 'COF': 0, 'CAH': 0, 'KMX': 0, 'CCL': 0, 'CARR': 0, 'CTLT': 0, 'CAT': 0, 'CBOE': 0, 'CBRE'

,^GSPC
Date,
2021-06-01,0.000000
2021-06-02,0.001447
2021-06-03,-0.002187
2021-06-04,0.006628
2021-06-07,0.005826
2021-06-08,0.006002
2021-06-09,0.004167
2021-06-10,0.008839
2021-06-11,0.010804


idx: 2021-06-01 00:00:00
self.df


...

,VLO,IQV,BALL,HRL,PPL,ABT,PNW,PSA,RCL,AMCR,AON,LMT,SJM,LNT,L,SYY,TXN,DLTR,TYL,REG,EA,MRK,BKR,FDX,HWM,DOV,DG,ADSK,UHS,TGT,ZBH,LKQ,AAPL,LRCX,BDX,ZBRA,COF,CVS,DTE,AVY,CDAY,TXT,AMP,HCA,AWK,MSCI,AMT,INCY,TT,NDAQ,TRGP,GE,DD,RF,WST,ORCL,SBNY,AMAT,GLW,PXD,NSC,AKAM,ABC,CPRT,HII,PEP,ROL,WEC,WTW,META,AVB,WMB,DPZ,NOC,DISH,STE,WYNN,AMZN,SEDG,MTCH,TMO,STZ,APTV,PEAK,DOW,BAX,FTV,UDR,HD,MA,CHD,CMCSA,PH,CTRA,ED,PWR,BEN,FIS,MNST,DLR,ETR,UAL,DE,CTLT,TECH,GPC,CMI,CB,KLAC,ALGN,MMM,NTRS,EMR,ALLE,PNC,GEN,MPC,CNP,XOM,AIG,MAS,DHI,PGR,ON,O,PSX,KO,RE,SRE,PNR,ADI,NTAP,QRVO,LIN,KMB,DXC,OMC,PKG,SYK,PKI,CFG,TSN,RTX,ODFL,CL,BR,ADM,KDP,SWKS,RMD,PTC,LYB,TDG,EPAM,ACN,V,VFC,WM,EBAY,AZO,AES,CF,KHC,GS,NI,POOL,REGN,MKC,RL,IR,LW,ETSY,MRNA,EXPE,CMG,GD,ADBE,HOLX,ROST,WRK,ZTS,C,CHRW,J,KR,AME,TROW,SNPS,ARE,AFL,LYV,COST,NWL,EFX,ROK,FE,MU,ISRG,EQT,WDC,LNC,BKNG,FMC,WBD,UPS,UNH,WY,LLY,EXPD,GIS,TSCO,KMX,RJF,AVGO,SBUX,HAL,CSCO,TSLA,DRI,BLK,ALB,SCHW,PAYC,NWS,CME,GL,CSX,CINF,EMN,D,TEL,FISV,SEE,RSG,AAP,EOG,CRM,HBAN,ZION,COP,IDXX,TAP,INVH,CHTR,TDY,ECL,BA,EXC,CCL,XEL,AJG,AIZ,AOS,EW,BBY,ETN,MCD,ATVI,K,LH,CTAS,PRU,SBAC,TFC,MHK,DXCM,LUMN,NFLX,WAB,CDNS,MCHP,MDT,SHW,KMI,EQIX,TFX,VNO,KEYS,HPE,COO,WBA,PYPL,CNC,PHM,GRMN,WELL,YUM,IP,ACGL,WFC,CVX,FTNT,DVN,JNPR,BXP,VZ,USB,NEE,CE,FOXA,NRG,MSI,ORLY,FLT,HSIC,CPT,PG,HAS,HUM,F,MOH,SLB,FAST,FFIV,TRV,RHI,ANET,HES,ELV,IT,NOW,IBM,DAL,GOOGL,CAG,FDS,EIX,WMT,TRMB,INTC,BRO,ITW,LDOS,CAT,IEX,EVRG,LOW,PLD,SYF,HST,ALK,ILMN,GILD,NXPI,BBWI,VTR,FOX,DGX,NUE,MMC,A,ALL,CZR,VMC,BAC,AAL,ABBV,DHR,CMS,NVR,ICE,WHR,JBHT,OXY,MS,DUK,DVA,PPG,BSX,GPN,CLX,MOS,CDW,LUV,APA,PEG,NKE,IVZ,VRSN,MPWR,NVDA,CPB,AEE,APH,TJX,GM,JPM,ES,SPGI,SPG,GWW,LVS,HLT,DIS,EQR,XRAY,ENPH,NWSA,IPG,LHX,URI,CRL,MDLZ,MCK,IFF,VTRS,MTB,GNRC,OKE,MO,BIO,CSGP,CTVA,FITB,IRM,HON,OTIS,LEN,NCLH,QCOM,ROP,TMUS,KEY,PM,GOOG,MLM,SO,CMA,VRTX,MAA,HIG,TPR,NDSN,MAR,TER,VICI,FANG,AXP,MRO,PFG,WRB,PCAR,MGM,JNJ,PFE,INTU,KIM,CARR,EXR,STT,CBOE,ATO,BWA,EL,WAT,XYL,CCI,CTSH,MKTX,FRT,JKHY,BIIB,PAYX,PCG,AEP,STX,BK,HPQ,CBRE,SWK,JCI,ANSS,APD,CAH,FRC,MCO,SNA,AMGN,NEM,MSFT,MTD,ULTA,DFS,HSY,CI,T,BMY,PARA,ADP,ESS,TTWO,ABMD,VRSK,FCX,SIVB,AMD,UNP,MET,FBHS
VLO,1.000000,-0.381837,0.091077,-0.055686,-0.210827,-0.491022,-0.213152,0.244892,0.541960,0.892125,0.412669,0.522857,-0.467256,-0.561465,0.509237,0.350204,-0.074332,-0.158210,-0.084582,0.700212,-0.157623,-0.079141,0.596028,0.326322,0.585356,0.657736,-0.047480,-0.161392,0.511447,-0.354188,-0.151277,0.648504,0.043057,-0.121534,-0.587367,-0.026539,0.768148,-0.632664,0.297420,0.646479,0.238421,0.694118,0.794028,0.392804,-0.489479,0.066464,0.207017,-0.104632,0.618217,0.172980,0.755496,0.498323,0.651800,0.868909,-0.246830,0.594635,0.708142,-0.065736,0.205433,0.794799,0.567609,-0.021552,-0.011786,0.002028,0.755219,-0.210904,0.195672,-0.126977,0.269166,0.256822,0.339966,0.658653,-0.107162,0.231619,0.759549,0.292806,0.448027,0.140422,-0.451970,0.122217,-0.295986,-0.078993,0.713979,0.122672,0.780432,-0.062630,0.367204,0.452127,0.223219,0.071934,-0.397143,0.258550,0.784796,-0.035429,-0.408191,0.670799,0.757350,0.045238,0.059605,-0.459348,-0.133765,0.707449,0.733370,-0.009871,-0.340157,0.655120,0.684381,0.600714,-0.242310,0.039531,0.701544,0.584175,0.788548,0.166427,0.712139,0.358687,0.796928,0.355236,0.793785,0.783889,0.449803,0.506965,0.522863,0.107700,0.013385,0.815788,0.592762,0.496446,0.431696,0.571349,-0.355827,0.325336,-0.024608,0.580099,0.212064,0.398254,0.657895,0.788029,-0.120927,-0.077264,0.849521,0.220453,0.694770,0.772363,-0.404741,0.372883,0.498603,0.255985,-0.147538,0.139175,-0.114656,0.920602,0.686871,-0.163526,-0.032539,0.072008,0.111880,0.332714,0.109321,0.697608,0.369111,0.767257,0.291830,0.828585,0.452189,0.424883,0.248348,-0.585052,0.487566,0.784858,0.162210,-0.189053,0.216911,0.211622,0.210361,0.646142,-0.258786,-0.211973,0.147941,0.815360,-0.456568,0.701365,0.547909,0.439766,0.451433,0.719246,0.228327,-0.198800,0.045654,0.823692,0.029496,0.312837,0.727272,0.314580,0.792447,0.477321,0.485647,-0.329856,0.338343,0.645178,0.665125,-0.295014,0.743161,0.456240,0.311016,-0.413266,0.75201

correlation_matrix sum: 46149.40860813156


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



returns: QuadraticModel({'VLO': 1.7509688617965808, 'IQV': -2.8747433307287857, 'BALL': 0.059056451459980425, 'HRL': -0.19598934428447018, 'PPL': -0.10502270698623302, 'ABT': -9.612547630896886, 'PNW': -0.12536396290313986, 'PSA': 3.0559714274904675, 'RCL': 3.0752153257012127, 'AMCR': 0.0854638409813183, 'AON': -0.12850459095176175, 'LMT': -0.2736336090875095, 'SJM': 0.21542758977955478, 'LNT': -0.15267424990401052, 'L': 0.11985185751184298, 'SYY': 0.44452591743046455, 'TXN': -0.8119374784055797, 'DLTR': 1.399529483360964, 'TYL': -1.1566660083642484, 'REG': 1.7817271703121333, 'EA': 0.8651813407961033, 'MRK': -0.6533224379154027, 'BKR': 1.1235690356210783, 'FDX': -3.877634278598363, 'HWM': 0.1302247005490614, 'DOV': 0.6269573969823247, 'DG': -1.51960099603525, 'ADSK': -4.487257130412426, 'UHS': 1.1028019686099957, 'TGT': 0.3466642636045684, 'ZBH': -2.012569109186664, 'LKQ': 0.1907108583683523, 'AAPL': -0.3262592402058698, 'LRCX': -0.8074029705154553, 'BDX': -3.211540642605931, 'ZBRA': 

SolverFailureError: ignored